## Problem description

The goal here is to solve a simple scenarios based asset allocation problem. We have m = 3 scenarios and n = 4 assets. The details are as follows:
+ Cost of assets: $c = [2.0, 3.0, 1.0, 5.0]$
Payoffs of the n assets and the requirements in the m scenarios
+ Scenario 1: Payoff $S_1 = [0.2,  1. ,  0.1,  0.5]$ Requirement $r_1 = 10.0$
+ Scenario 2: Payoff $S_2 = [0.5,  1.2,  1. ,  0.8]$ Requirement $r_2 = 20.0$
+ Scenario 3: Payoff $S_3 = [1. ,  0.2,  1.3,  1.2]$ Requirement $r_3 = 15.0$

Linear programming formulation: $\phi$ = number of shares in assets

$$
\begin{array}{rl}
\mbox{min} & \sum_{i=1}^n c_i \phi_i\\
\mbox{subject to} & \sum_{i = 1}^n S_{ji}\phi_i >= r_j, \quad j = 1, \ldots, m
\end{array}
$$

### Load up numpy and cvxpy

In [1]:
import numpy as np
import cvxpy as cvx
import gurobipy as grb

### Set up the data for the problem

In [2]:
numassets = 4;
numscenarios = 3;
S = np.matrix([
    [ 0.2,  1. ,  0.1,  0.5],
    [ 0.5,  1.2,  1. ,  0.8],
    [ 1. ,  0.2,  1.3,  1.2]])
r = np.matrix([                         #column vector
    [10.0],
    [20.0],
    [15.0]]);
cost = np.matrix([2.0,3.0,1.0,5.0]);    #row vector

### Model formulation
A CVXPY model consists of three components
+ Variables: $\phi$
+ Objective function: $\mbox{minimize}\ c'\phi$
+ Constraints: $S\phi \geq r$

In [3]:
phi = cvx.Variable(numassets);               # Variable is a function defined in the cvxpy package. The argument is
                                             # the dimension of the variable vector. By convention, the cvx package
                                             # assumes that variables are all column vectors. Here, phi is a column
                                             # vector with dimension 4.
objective = cvx.Minimize(cost*phi);          # To avoid using transpose, cost was defined as a row vector.
constraints = []
for j in range(numscenarios):
    constraints += [S[j,:]*phi >= r[j]];
constraints += [phi >= 0]                    # Unlike linprog, cvx does not assume that variables are nonnegative.
prob = cvx.Problem(objective, constraints)   # phi contains decision variables

### Solve the problem
And print solution in case the problem is solved to optimality

In [4]:
prob.solve()                                              # Unlike linprog and Excel, which use the Simplex algorithm
                                                          # to solve LPs, cvxpy uses Interior Point algorithms.

print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: ' + str(prob.value));
    print('Variable values: ')
    print(phi.value)

Problem status: optimal
Problem value: 37.109375000000014
Variable values: 
[5.71896098e-23 8.98437500e+00 1.01562500e+01 4.08879600e-23]


Solve the problem using a different solver

In [5]:
print(cvx.installed_solvers())                         # GUROBI gives you the option to use either Simplex or Interior
                                                       # Point methods.
prob.solve(solver=cvx.GUROBI)
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: ' + str(prob.value));
    print('Variable values: ')
    print(phi.value)

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'OSQP', 'SCS']
Using license file /Users/joehigh/gurobi.lic
Problem status: optimal
Problem value: 37.109375
Variable values: 
[ 0.        8.984375 10.15625   0.      ]


Can you explain the difference in the solutions? 

Why only two assets purchased?

Lets analyze the constraints

In [6]:
print(np.concatenate((S*phi.value, r), axis=1))

ValueError: shapes (3,4) and (1,4) not aligned: 4 (dim 1) != 1 (dim 0)

Does this give a clue to why only two assets?

Plot the portfolio

In [7]:
import matplotlib.pyplot as plt
plt.plot(phi.value)
plt.show()

<Figure size 640x480 with 1 Axes>